<a href="https://colab.research.google.com/github/bhashitha69/TRMIMO-MPC/blob/main/TRMIMO_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.linalg import matrix_power
from numpy.linalg import inv
from scipy.linalg import null_space
from sympy import Matrix

In [2]:
c = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9]])
c = c.T
c[2][0]

3

In [4]:
def A_trmimo(X,U):
  #input X as 7x1 vector
  #input U as 2x1 vector

  u1 = U[0][0];                  #rotor 1 input
  u2 = U[1][0];                  #rotor 2 input

  x1 = X[0][0];                  #pitch displacement
  x2 = X[1][0];                  #angular velocity of pitch axis
  x3 = X[2][0];                  #yaw displacement
  x4 = X[3][0];                  #angular velocity of yaw axis
  x5 = X[4][0];                  #angular velocity of rotor 1(thrust toque on pitch axis)
  x6 = X[5][0];                  #angular velocity of rotor 2(thrust toque on yaw axis)
  x7 = X[6][0];                  #cross affect on pitch axis from rotor 2

  I_1 = 0.068;                   #moment of inertia of vertical rotor
  I_2 = 0.02;                    #moment of inertia of horizontal rotor
  a_1 = 0.0135;                  #static characteristic parameter
  b_1 = 0.0924;                  #static characteristic parameter
  a_2 = 0.02;                    #static characteristic parameter
  b_2 = 0.09;                    #static characteristic parameter
  M_g = 0.32;                    #gravity momentum
  B_1psi = 0.006;                #friction momentum function parameter
  B_2psi = 0.001;                #friction momentum function parameter
  B_1phi = 0.1;                  #friction momentum function parameter
  B_2phi = 0.01;                 #friction momentum function parameter
  K_gy = 0.05;                   #gyroscopic momentum parameter
  k_1 = 1.1;                     #motor 1 gain
  k_2 = 0.8;                     #motor 2 gain
  T_11 = 1.1;                    #motor 1 denominator parameter
  T_10 = 1;                      #motor 1 denominator parameter
  T_21 = 1;                      #motor 2 denominator parameter
  T_20 = 1;                      #motor 2 denominator parameter
  T_p = 2;                       #cross reaction momentum parameter
  T_o = 3.5;                     #cross reaction momentum parameter
  k_c = -0.2;                    #cross reaction momentum gain


  dx1 = x2;
  dx2 = (a_1*x5**2 + b_1*x5 - M_g*np.sin(x1) - B_1psi*x2 - (B_2psi*abs(x2))/x2 - K_gy*(a_1*x5 + b_1)*x5*x4*np.cos(x1))/I_1;
  dx3 = x4;
  dx4 = ((a_2*x6 + b_2)*x6 - B_1phi*x4 - (B_2phi*abs(x4))/x4 - x7)/I_2;
  dx5 = (k_1*u1 - T_10*x5)/T_11;
  dx6 = (k_2*u2 - T_20*x6)/T_21;
  dx7 = (k_c*x5 - x7 + k_c*T_o*x7)/T_p;

  return np.array([[dx1, dx2, dx3, dx4, dx5, dx6, dx7]]).T  # Output as 7x1 vector

In [26]:
T = 8; dT = 0.01;
timerange = np.arange(0,T,dT);
vol1 = 2.5*np.ones((200,),dtype = int);
vol1 = np.array([vol1]);
vol2 = np.zeros((200,),dtype = int);
vol2 = np.array([vol2]);
vol3 = np.concatenate((vol1,vol2,vol1,vol2), axis = 1);
vol4 = np.zeros((100,),dtype = int);
vol4 = np.array([vol4]);
vol5 = np.concatenate((vol4,vol1,vol2,-vol1,vol4), axis = 1);

In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=timerange, y=vol5[0], mode='lines', name='$V_{in}\\text{ of 2nd motor}$', marker=dict(color="blue")));
fig.add_trace(go.Scatter(x=timerange, y=vol3[0], mode='lines', name='$V_{in}\\text{ of 1st motor}$', marker=dict(color="red")))

In [33]:
U = np.concatenate((vol3,vol5),axis = 0);
U[:,259]

array([0. , 2.5])

In [34]:
X = np.array([[np.pi/8, 0, 0, 0, 0, 0, 0]]).T;
U = np.concatenate((vol3,vol5), axis = 0);
dt = 0.02;
x = X;
OutX = X;
for i in range(0,802,2):
  uI1 = np.array([U[:,i]]).T;
  uI2 = np.array([U[:,i + 1]]).T;

  uIf = np.array([U[:,i + 2]]).T;

  K1 = dt*A_trmimo(x,uI1);
  K2 = dt*A_trmimo(x + K1/2,uI2);
  K3 = dt*A_trmimo(x + K2/2,uI2);
  K4 = dt*A_trmimo(x + K3,uIf);

  x = x + K1/6 + K2/3 + K3/3 + K4/6;
  OutX = np.concatenate((OutX,x), axis = 1)

<ipython-input-4-292d80523706>:40: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-4-292d80523706>:42: RuntimeWarning:

invalid value encountered in double_scalars



IndexError: ignored